In [1]:
import json

def preprocess_and_segment_stories(input_file, output_file):
    # 1. Đọc nội dung file thô
    with open(input_file, 'r', encoding='utf-8') as f:
        full_text = f.read()

    # 2. Tách các truyện dựa trên dấu phân cách ("="*80)
    story_blocks = full_text.split("="*80)
    
    dataset_squad_format = {"data": [], "version": "1.0_viefolktale"}
    story_counter = 0

    for block in story_blocks:
        block = block.strip()
        if not block:
            continue

        # 3. Trích xuất Tiêu đề và Link từ mỗi block
        lines = block.split('\n')
        
        # Bỏ qua dòng tiêu đề chung (KHO TÀNG...)
        if lines[0].startswith("KHO TÀNG TRUYỆN CỔ TÍCH"):
            continue

        title = "Không rõ tiêu đề"
        link = "Không rõ link"
        content_start_index = -1
        
        for i, line in enumerate(lines):
            line = line.strip()
            if line.startswith("[") and line.endswith("]"):
                # Dòng chứa [idx] Tiêu đề
                title = line.split("]")[0].split("[")[1] + "] " + line.split("]")[1].strip()
            elif line.startswith("Link:"):
                link = line.replace("Link:", "").strip()
            elif line.startswith("-" * 60):
                content_start_index = i + 1
                break
        
        if content_start_index == -1:
             continue # Không tìm thấy nội dung

        # 4. Tách nội dung thành các đoạn (paragraphs)
        raw_content = '\n'.join(lines[content_start_index:]).strip()
        
        # Tách dựa trên dòng trống (\n\n) - tiêu chuẩn của đoạn văn
        paragraphs = [p.strip() for p in raw_content.split('\n\n') if p.strip()]

        story_entry = {
            "title": title,
            "link": link,
            "paragraphs": []
        }

        # 5. Xử lý từng đoạn văn
        for p_idx, context in enumerate(paragraphs):
            # Cần làm sạch thêm (ví dụ: loại bỏ khoảng trắng thừa)
            context = ' '.join(context.split()) 
            
            # Chỉ lấy đoạn có độ dài hợp lý
            if len(context.split()) < 5:
                continue

            # Chuẩn bị cấu trúc cho việc ghi nhãn
            story_entry["paragraphs"].append({
                "context": context,
                "qas": []  # Q&A sẽ được thêm vào thủ công sau
            })
        
        if story_entry["paragraphs"]:
            dataset_squad_format["data"].append(story_entry)
            story_counter += 1

    # 6. Lưu file JSON đã được phân đoạn
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(dataset_squad_format, f, ensure_ascii=False, indent=4)
        
    print(f"Hoàn tất tiền xử lý. Đã tạo {story_counter} truyện với tổng số {sum(len(s['paragraphs']) for s in dataset_squad_format['data'])} đoạn văn (Context).")

# Tên file của bạn
input_file_name = "Tat_Ca_Truyen_Co_Tich_Lay_Thu_Cong.txt"
output_file_name = "vietnamese_folktale_preprocessed.json"

preprocess_and_segment_stories(input_file_name, output_file_name)

Hoàn tất tiền xử lý. Đã tạo 119 truyện với tổng số 2313 đoạn văn (Context).
